# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = 'temp'
assert FOLDERNAME is not None

import sys
sys.path.append('/content/drive/My\ Drive/{}'.format(FOLDERNAME))
%cd /content/drive/My Drive/temp

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/temp


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms

## Load Data

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 32

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)

split_ratio = 0.8
total_size = len(trainset)
train_size = int(split_ratio * total_size)
valid_size = total_size - train_size

train_dataset, valid_dataset = torch.utils.data.random_split(trainset, [train_size, valid_size])

trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
validloader = torch.utils.data.DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)

testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


# Train Classifier

## Training Loop

In [ ]:
def train(model, epochs):
    for epoch in range(epochs):

        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad()

            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            if i % 100 == 99:
                print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / 2000:.3f}')
                running_loss = 0.0

        valid_acc = validate(model)
        print(f'accuracy: {valid_acc}')

def validate(model):
    correct = 0
    total = 0

    with torch.no_grad():
        for data in validloader:
            inputs, labels = data
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)

            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    acc = (100 * correct // total)
    return (acc)

## Hyperparameter Grid

In [ ]:
from itertools import product

num_filters = [64]
learning_rates = [5e-4]

hyperparameter_grid = product(num_filters, learning_rates)

## Grid Search

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from cnn import CNN

criterion = nn.CrossEntropyLoss()

best_acc = 0
best_model = None
best_params = None, None

for num_filter, learning_rate in hyperparameter_grid:

    print(f"num_filter={num_filter}, learning_rate={learning_rate}")
    model = CNN(10, num_filter).to(device)
    optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=.9)
    train(model, 20)
    valid_acc = validate(model)

    if(valid_acc > best_acc):
        best_model = model
        best_acc = valid_acc
        best_params = num_filter, learning_rate

print(best_acc)
print(best_params)

num_filter=64, learning_rate=0.0005
[1,   100] loss: 0.094
[1,   200] loss: 0.078
[1,   300] loss: 0.072
[1,   400] loss: 0.067
[1,   500] loss: 0.065
[1,   600] loss: 0.063
[1,   700] loss: 0.059
[1,   800] loss: 0.058
[1,   900] loss: 0.057
[1,  1000] loss: 0.055
[1,  1100] loss: 0.051
[1,  1200] loss: 0.052
accuracy: 64
[2,   100] loss: 0.044
[2,   200] loss: 0.043
[2,   300] loss: 0.043
[2,   400] loss: 0.043
[2,   500] loss: 0.044
[2,   600] loss: 0.042
[2,   700] loss: 0.043
[2,   800] loss: 0.040
[2,   900] loss: 0.041
[2,  1000] loss: 0.041
[2,  1100] loss: 0.038
[2,  1200] loss: 0.040
accuracy: 71
[3,   100] loss: 0.030
[3,   200] loss: 0.030
[3,   300] loss: 0.030
[3,   400] loss: 0.032
[3,   500] loss: 0.031
[3,   600] loss: 0.031
[3,   700] loss: 0.030
[3,   800] loss: 0.031
[3,   900] loss: 0.031
[3,  1000] loss: 0.030
[3,  1100] loss: 0.031
[3,  1200] loss: 0.031
accuracy: 74
[4,   100] loss: 0.023
[4,   200] loss: 0.023
[4,   300] loss: 0.021
[4,   400] loss: 0.022
[4,  

## Accuracy on Test Data

In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct // total} %')

Accuracy: 78 %


In [ ]:
model_path = 'raw_model.pth'
torch.save(best_model, model_path)

In [ ]:
model_path = 'raw_model.pth'
model = CNN(10, 64).to(device)
model.load_state_dict(torch.load(model_path))